<a href="https://colab.research.google.com/github/Randasabag/Projet-5-ML/blob/main/P5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

La table `Posts` dans la base de données de Stack Exchange contient les informations de tous les messages publiés sur la plateforme, incluant les questions, les réponses et les messages wiki. Voici une explication des champs courants dans cette table :

1. **Id** :
   - **Type** : Integer
   - **Description** : Identifiant unique du post.

2. **PostTypeId** :
   - **Type** : Integer
   - **Description** : Identifie le type de post. Les valeurs communes sont :
     - 1 : Question
     - 2 : Réponse

3. **ParentId** :
   - **Type** : Integer
   - **Description** : Identifiant de la question parente si le post est une réponse. Null pour les questions.

4. **AcceptedAnswerId** :
   - **Type** : Integer
   - **Description** : Identifiant de la réponse acceptée pour les questions. Null si aucune réponse n'est acceptée ou si le post est une réponse.

5. **CreationDate** :
   - **Type** : DateTime
   - **Description** : Date et heure de la création du post.

6. **Score** :
   - **Type** : Integer
   - **Description** : Score du post, calculé comme le nombre de votes positifs moins le nombre de votes négatifs.

7. **ViewCount** :
   - **Type** : Integer
   - **Description** : Nombre de fois que la question a été vue. Applicable uniquement aux questions.

8. **Body** :
   - **Type** : Text
   - **Description** : Contenu du post en HTML.

9. **OwnerUserId** :
   - **Type** : Integer
   - **Description** : Identifiant de l'utilisateur qui a créé le post. Null pour les posts anonymes.

10. **OwnerDisplayName** :
    - **Type** : Text
    - **Description** : Nom d'affichage de l'utilisateur pour les posts anonymes.

11. **LastEditorUserId** :
    - **Type** : Integer
    - **Description** : Identifiant de l'utilisateur qui a effectué la dernière modification du post.

12. **LastEditDate** :
    - **Type** : DateTime
    - **Description** : Date et heure de la dernière modification du post.

13. **LastActivityDate** :
    - **Type** : DateTime
    - **Description** : Date et heure de la dernière activité sur le post (édition, réponse, commentaire, etc.).

14. **Title** :
    - **Type** : Text
    - **Description** : Titre du post. Applicable uniquement aux questions.

15. **Tags** :
    - **Type** : Text
    - **Description** : Liste des tags associés au post. Les tags sont encadrés par des chevrons (par exemple, `<python><pandas>`).

16. **AnswerCount** :
    - **Type** : Integer
    - **Description** : Nombre de réponses associées à une question. Applicable uniquement aux questions.

17. **CommentCount** :
    - **Type** : Integer
    - **Description** : Nombre de commentaires associés au post.

18. **FavoriteCount** :
    - **Type** : Integer
    - **Description** : Nombre de fois que la question a été mise en favori par des utilisateurs. Applicable uniquement aux questions.

19. **ClosedDate** :
    - **Type** : DateTime
    - **Description** : Date et heure à laquelle la question a été fermée. Null si la question n'est pas fermée.

20. **CommunityOwnedDate** :
    - **Type** : DateTime
    - **Description** : Date et heure à laquelle le post est devenu « community-owned », ce qui signifie qu'il appartient à la communauté et non à un utilisateur spécifique.

Ces champs fournissent une riche source d'information sur chaque post, permettant une analyse détaillée des contributions et des interactions des utilisateurs sur la plateforme Stack Exchange.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as st
from scipy.stats import shapiro
import statsmodels.api as sm
import statsmodels.formula.api as smf
from datetime import datetime,date,timedelta
import datetime as dt
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
df=pd.read_csv('/content/drive/MyDrive/MACHINE_LEARNING/Requete principaleP5.csv', sep=',')
df.head()

In [ ]:
print(df.shape)

In [ ]:
df.isnull().sum()

In [ ]:
df.duplicated().sum()

In [ ]:
df.dtypes

In [ ]:
df['FavoriteCount'].value_counts()

In [ ]:
df['Score'].value_counts()

In [ ]:
df['ViewCount'].value_counts()

In [ ]:
#Résumé stat
df.describe(include="all").round(2)

In [ ]:
df.columns

In [ ]:
var_num = ['Score', 'ViewCount', 'FavoriteCount', 'AnswerCount']

In [ ]:
#Visualisation des outliers
plt.figure(figsize=(20, 10))
sns.set(style="white", palette="husl")

#Création des boxplots pour chaque variable
for i, var in enumerate(var_num, 1):
  plt.subplot(6, 6, i)
  sns.boxplot(data=df, x=var)

#Espacement entre les graphiques
plt.tight_layout()

plt.savefig("boxplots_outliers.png")

In [ ]:
#Visualisation des histogrammes de chaque variable
plt.figure(figsize=(20, 10))
sns.set(style="white", palette="husl")
for i, var in enumerate(var_num, 1):
    plt.subplot(6, 6, i)
    sns.histplot(data=df, x=var)

#Espacement entre les graphiques
plt.tight_layout()

plt.savefig("hist.png")

In [ ]:
# Visualisation de la variable cible
fig = plt.figure(figsize=(8,4))
ax = sns.histplot(data=df['Score'])
ax.set_xlabel("Score")
plt.savefig("score.png")
plt.show()

In [ ]:
# Transformation de la variable cible en log pour suivre une loi normale et changer son échelle
ylog_train = np.log1p(df['Score'])
fig = plt.figure(figsize=(8,4))
ax = sns.histplot(data=ylog_train)
ax.set_xlabel("Score")
plt.savefig("scorelog.png")
plt.show()

In [ ]:
# Visualisation de la variable cible
fig = plt.figure(figsize=(8,4))
ax = sns.histplot(data='ViewCount')
ax.set_xlabel("ViewCount")
plt.savefig("viewcount.png")
plt.show()

In [ ]:
sns.pairplot(df[var_num].dropna(), diag_kind="kde", height=1.5, x_vars=var_num, y_vars=var_num, corner=True, kind='reg')
plt.tight_layout()
plt.savefig("pairplot.png")

In [ ]:
plt.figure(figsize=(40, 15))
df_finum = df_sans_outliers[vnum].dropna()
matrix = df_finum.corr()

# Créez un masque pour masquer la moitié inférieure de la matrice
mask = np.triu(np.ones_like(matrix, dtype=bool))

sns.heatmap(data=matrix, annot=True, cmap='PiYG', mask=mask)
plt.xticks(rotation=10)
plt.tight_layout()
plt.savefig("matrice_corr.png")
plt.show()